<a href="https://colab.research.google.com/github/TastyButSadly/Notebooks/blob/main/Possum_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
!pip install opendatasets --quiet
import opendatasets as od



In [5]:
od.download('https://www.kaggle.com/datasets/abrambeyer/openintro-possum/data')

Skipping, found downloaded files in "./openintro-possum" (use force=True to force download)


In [7]:
df = pd.read_csv('/content/openintro-possum/possum.csv')
df.head()

,case,site,Pop,sex,age,hdlngth,skullw,totlngth,taill,footlgth,earconch,eye,chest,belly
0,1,1,Vic,m,8.0,94.1,60.4,89.0,36.0,74.5,54.5,15.2,28.0,36.0
1,2,1,Vic,f,6.0,92.5,57.6,91.5,36.5,72.5,51.2,16.0,28.5,33.0
2,3,1,Vic,f,6.0,94.0,60.0,95.5,39.0,75.4,51.9,15.5,30.0,34.0
3,4,1,Vic,f,6.0,93.2,57.1,92.0,38.0,76.1,52.2,15.2,28.0,34.0
4,5,1,Vic,f,2.0,91.5,56.3,85.5,36.0,71.0,53.2,15.1,28.5,33.0


In [9]:
df.columns

Index(['case', 'site', 'Pop', 'sex', 'age', 'hdlngth', 'skullw', 'totlngth',
       'taill', 'footlgth', 'earconch', 'eye', 'chest', 'belly'],
      dtype='object')

In [89]:
df.drop(columns=['case', 'site', 'Pop', 'sex'])

KeyError: "['case', 'site', 'Pop', 'sex'] not found in axis"

In [14]:
df.shape

(104, 14)

In [23]:
df.isna().sum()

,0
case,0
site,0
Pop,0
sex,0
age,2
hdlngth,0
skullw,0
totlngth,0
taill,0
footlgth,1


In [42]:
df.dropna(inplace=True)
df.drop(columns=['case', 'site', 'Pop', 'sex'], inplace=True)

In [90]:
X = df.drop(columns=['age']).values
y = df['age'].values

In [54]:
X_train, X_test, y_train, y_test = train_test_split(x, y, shuffle=True, test_size=0.2, random_state=20)

In [45]:
def euclid_dist(v1, v2):
  return np.linalg.norm(v1 - v2)

In [36]:
df.dtypes

,0
case,int64
site,int64
Pop,object
sex,object
age,float64
hdlngth,float64
skullw,float64
totlngth,float64
taill,float64
footlgth,float64


In [46]:
def get_neighbors(train, test, k):
  dist = np.linalg.norm(train - test, axis=1)
  return dist.argsort()[:k]

In [47]:
get_neighbors(X_train[:5], X_test[1], 3)

array([1, 3, 4])

Предсказания для тестовых данных

In [62]:
def predict(X_train, X_test, y_train, k):
  y_res = []
  for x_test in X_test:
    nn_id = get_neighbors(X_train, x_test, k)
    nn_y = y_train[nn_id]
    y_res.append(np.mean(nn_y))
  return y_res

In [63]:
y_predict = predict(X_train[:30], X_test[:5], y_train[:30], k = 5)
y_predict

[5.0, 5.2, 5.8, 4.4, 5.0]

Через Sklearn

In [68]:
model = KNeighborsRegressor(n_neighbors=8)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

y_pred

array([3.875, 4.875, 5.625, 3.75 , 5.25 , 1.875, 5.   , 2.25 , 4.125,
       4.625, 4.875, 4.875, 4.25 , 2.25 , 5.   , 2.125, 4.   , 4.5  ,
       3.   , 2.625, 3.75 ])

In [84]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [69]:
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

MSE_train = mean_squared_error(y_train, pred_train)
RMSE_train = np.sqrt(MSE_train)
R2_train = r2_score(y_train, pred_train)
MAE_train = mean_absolute_error(y_train, pred_train)

MSE_test = mean_squared_error(y_test, pred_test)
RMSE_test = np.sqrt(MSE_test)
R2_test = r2_score(y_test, pred_test)
MAE_test = mean_absolute_error(y_test, pred_test)

print(f'MSE на обучающей {MSE_train:.2f}')
print(f'MSE на тесте {MSE_test:.2f}', end='\n\n')

print(f'RMSE на обучающей {RMSE_train:.2f}')
print(f'RMSE на тесте {RMSE_test:.2f}', end='\n\n')

print(f'R2 на обучающей {R2_train:.2f}')
print(f'R2 на тесте {R2_test:.2f}', end='\n\n')

print(f'MAE на обучающей {MAE_train:.2f}')
print(f'MAE на тесте {MAE_test:.2f}', end='\n\n')

MSE на обучающей 2.48
MSE на тесте 3.72

RMSE на обучающей 1.58
RMSE на тесте 1.93

R2 на обучающей 0.32
R2 на тесте -0.04

MAE на обучающей 1.32
MAE на тесте 1.39



In [81]:
from sklearn.tree import DecisionTreeRegressor

model_tree = DecisionTreeRegressor(random_state=20)
model_tree.fit(X_train, y_train)


pred_train = model_tree.predict(X_train)
pred_test = model_tree.predict(X_test)

In [82]:
MSE_train = mean_squared_error(y_train, pred_train)
RMSE_train = np.sqrt(MSE_train)
R2_train = r2_score(y_train, pred_train)
MAE_train = mean_absolute_error(y_train, pred_train)

MSE_test = mean_squared_error(y_test, pred_test)
RMSE_test = np.sqrt(MSE_test)
R2_test = r2_score(y_test, pred_test)
MAE_test = mean_absolute_error(y_test, pred_test)

print(f'MSE на обучении {MSE_train:.2f}')
print(f'MSE на тесте {MSE_test:.2f}', end='\n\n')

print(f'RMSE на обучении {RMSE_train:.2f}')
print(f'RMSE на тесте {RMSE_test:.2f}', end='\n\n')

print(f'R2 на обучении {R2_train:.2f}')
print(f'R2 на тесте {R2_test:.2f}', end='\n\n')

print(f'MAE на обучении {MAE_train:.2f}')
print(f'MAE на тесте {MAE_test:.2f}', end='\n\n')

MSE на обучении 0.00
MSE на тесте 6.62

RMSE на обучении 0.00
RMSE на тесте 2.57

R2 на обучении 1.00
R2 на тесте -0.85

MAE на обучении 0.00
MAE на тесте 2.24



In [88]:
X

array([[1, 1, 'Vic', ..., 15.2, 28.0, 36.0],
       [2, 1, 'Vic', ..., 16.0, 28.5, 33.0],
       [3, 1, 'Vic', ..., 15.5, 30.0, 34.0],
       ...,
       [102, 7, 'other', ..., 13.0, 25.0, 30.0],
       [103, 7, 'other', ..., 15.4, 25.0, 29.0],
       [104, 7, 'other', ..., 14.8, 28.5, 33.5]], dtype=object)

Кросс-валидация

In [91]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate

scores = cross_validate(DecisionTreeRegressor(random_state=44), X, y, cv=5,
                        scoring={'r2': make_scorer(r2_score),
                        'mean_squared_error': make_scorer(mean_squared_error)},
                        return_train_score=True)

print('R2 train mean = ', scores['train_r2'].mean())
print('R2 test mean = ', scores['test_r2'].mean())

print('MSE train mean = ', scores['train_mean_squared_error'].mean())
print('MSE test mean = ', scores['test_mean_squared_error'].mean())

R2 train mean =  1.0
R2 test mean =  -0.7644861990897471
MSE train mean =  0.0
MSE test mean =  5.855714285714286


-> не помогло

In [92]:
model_tree.get_params()

{'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'random_state': 20,
 'splitter': 'best'}

In [93]:
model_tree = DecisionTreeRegressor(random_state=1,
                              max_depth=4,
                              min_samples_leaf=2,
                              max_leaf_nodes=None)

In [99]:
model_tree.fit(X_train, y_train)

print(f'MSE train = {r2_score(y_train, model.predict(X_train))}')
print(f'MSE test = {r2_score(y_test, model.predict(X_test))}')

MSE train = 0.31639700503463997
MSE test = -0.04274539390088905


In [107]:
model_tree = DecisionTreeRegressor(random_state=1,
                              max_depth=4,
                              min_samples_leaf=1,
                              max_leaf_nodes=3)

model_tree.fit(X_train, y_train)

print(f'R2 = {r2_score(y_train, model.predict(X_train))}')
print(f'R2t = {r2_score(y_test, model.predict(X_test))}')

R2 = 0.31639700503463997
R2t = -0.04274539390088905


GridSearchCV

In [102]:
from sklearn.model_selection import GridSearchCV

In [104]:
model_grid = DecisionTreeRegressor()


param_grid = {
    'max_depth': np.arange(1, 5),
    'min_samples_leaf': [1,2, 3],
}

In [108]:
gridsearch = GridSearchCV(model_grid, param_grid, refit=True, scoring=make_scorer(r2_score))


gridsearch.fit(X_train, y_train)


print(gridsearch.best_params_)

best_model = gridsearch.best_estimator_

print(f'MSE train = {mean_squared_error(y_train, best_model.predict(X_train))}')
print(f'MSE test = {mean_squared_error(y_test, best_model.predict(X_test))}')

print(f'R2 train = {r2_score(y_train, best_model.predict(X_train))}')
print(f'R2 test = {r2_score(y_test, best_model.predict(X_test))}')

{'max_depth': 1, 'min_samples_leaf': 1}
MSE train = 2.399278846153846
MSE test = 3.169572414764722
R2 train = 0.3392407325881227
R2 test = 0.11195588633339137


save

In [109]:
import pickle

filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))


loaded_model = pickle.load(open(filename, 'rb'))